In [1]:
import os

import numpy as np
import pandas as pd

from sklearn import svm

from sklearn.feature_selection import mutual_info_classif, RFE, SequentialFeatureSelector, SelectFromModel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
data_dir = '/shared/rsaas/nschiou2/EROS/python/'

df = pd.read_parquet(os.path.join(data_dir, 'simple_bandpower_features.parquet'))

#### List subjects by ID

In [3]:
subjects = df['subject_id'].unique()
subjects

array(['127', '130', '146', '149', '150', '151', '152', '153', '154',
       '155', '157', '505', '516', '527', '534'], dtype=object)

#### Identify number of missing channels for each montage

In [4]:
iterables = [list(subjects), ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']]
missing_data = pd.MultiIndex.from_product(iterables, names=['subject_id', 'montage']).to_frame()

In [5]:
for s in subjects:
    subject_df = df[df['subject_id'] == s]
    for m in subject_df['montage'].unique():
        sub_sub_df = subject_df[subject_df['montage'] == m]
        non_null = subject_df.loc[:, sub_sub_df.isnull().sum(axis=0) != sub_sub_df.shape[0]]
        missing_data.loc[(s, m), 'num_avail_chan'] = (non_null.shape[1] - 4) / 3 / 7
        missing_data.loc[(s, m), 'num_null_chan'] = (sub_sub_df.loc[:, sub_sub_df.isnull().sum(axis=0) == sub_sub_df.shape[0]].shape[1]) / 3 / 7
    
missing_data.drop(['subject_id', 'montage'], axis=1, inplace=True)

In [36]:
pd.options.display.max_rows = 120
missing_data.groupby(['montage']).mean().T

montage,a,b,c,d,e,f,g,h
num_avail_chan,29.6,31.511111,29.733333,30.6,35.8,27.533333,26.866667,27.2
num_null_chan,591.4,589.488889,591.266667,590.4,585.2,593.466667,594.133333,593.8


In [38]:
label_df = df.copy()

def combine_motor_LR(x):
    """
    Combines trial types for left motor responses and right motor responses.
    
    0 represents left and 1 represents right.
    """
    if x in [2, 4, 6, 8]:
        return 0
    elif x in [1, 3, 5, 7]:
        return 1
    else:
        return np.nan

label_df.loc[:, 'label'] = label_df.loc[:, 'trial_type'].apply(combine_motor_LR)
label_df = label_df.dropna(axis=0, subset=['label']).copy().reset_index(drop=True)  # Remove vocal response trials for now

In [46]:
pd.DataFrame(label_df.groupby(['subject_id', 'montage']).count()['trial_num']).T

subject_id  127                                     130       ...  527       \
montage       a    b    c    d    e    f    g    h    a    b  ...    g    h   
trial_num   228  228  228  228  209  233  236  236  229  229  ...  221  233   

subject_id  534                                     
montage       a    b    c    d    e    f    g    h  
trial_num   227  227  219  231  210  233  227  227  

[1 rows x 120 columns]

### Choose example subject

In [24]:
subject_id = 127
montage = 'a'

subset = df[(df['subject_id'] == str(subject_id)) & (df['montage'] == montage)].copy().reset_index(drop=True)
print(f'Subject {subject_id} has {subset.shape[0]} original examples')

Subject 127 has 460 original examples


#### Identify discrete features and combine across labels

In [25]:
feat_cols = np.array([c for c in subset.columns if 'ph_' in c if f'_{montage}_' in c])
print(f'{len(feat_cols)} original features for montage {montage}')

1365 original features for montage a


In [26]:
discrete_feats = ((np.core.defchararray.find(feat_cols, 'samp_gt_zero') != -1) | 
                  (np.core.defchararray.find(feat_cols, 'zero_cross') != -1))
print(f'{np.count_nonzero(discrete_feats)} discrete features for montage {montage}')

390 discrete features for montage a


In [27]:
def combine_motor_LR(x):
    """
    Combines trial types for left motor responses and right motor responses.
    
    0 represents left and 1 represents right.
    """
    if x in [2, 4, 6, 8]:
        return 0
    elif x in [1, 3, 5, 7]:
        return 1
    else:
        return np.nan
    
subset.loc[:, 'label'] = subset.loc[:, 'trial_type'].apply(combine_motor_LR)
subset = subset.dropna(axis=0, subset=['label']).copy().reset_index(drop=True)  # Remove vocal response trials for now
print(f'Subject {subject_id} has {subset.shape[0]} motor L/R examples')

Subject 127 has 228 motor L/R examples


In [28]:
# Remove features that are based on zeroed channels across all trials
subset = subset.loc[:, subset.isnull().sum(axis=0) != subset.shape[0]]

# Keep track of features with zeroed channels for specific trials
na_mask = subset.isna().any(axis=0).values
na_cols = list(subset.columns[na_mask])
feat_cols = np.array([c for c in subset.columns if 'ph_' in c])
discrete_feats = ((np.core.defchararray.find(feat_cols, 'samp_gt_zero') != -1) | 
                  (np.core.defchararray.find(feat_cols, 'zero_cross') != -1))
print(f'{np.count_nonzero(na_mask)} / {len(feat_cols)} features are based on zeroed channels')
subset = subset.fillna(0)

21 / 441 features are based on zeroed channels


# Without Common Spatial Pattern (CSP) Filtering

### Train-test split for feature selection

In [75]:
X = subset[feat_cols].values
y = subset['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [76]:
def train_eval_clf(X_train, y_train, X_test, y_test):
    
    # scaler = StandardScaler()
    # scaler.fit(X_train)
    # X_train = scaler.transform(X_train)
    # X_test = scaler.transform(X_test)
    
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy: \t{:.3f}'.format(accuracy_score(y_test, y_pred)))
    print('Precision: \t{:.3f}'.format(precision_score(y_test, y_pred)))
    print('Recall: \t{:.3f}'.format(recall_score(y_test, y_pred)))
    print('F1-Score: \t{:.3f}'.format(f1_score(y_test, y_pred)))

### PCA

In [77]:
pca = PCA(n_components=100)
pca.fit(X_train)
exp_var = pca.explained_variance_ratio_

In [78]:
train_eval_clf(pca.transform(X_train), y_train, pca.transform(X_test), y_test)

Accuracy: 	0.674
Precision: 	0.778
Recall: 	0.560
F1-Score: 	0.651


### Recursive Feature Elimination

Moved to script rfe.py

In [353]:
support = np.load(os.path.join('RFE', '300_features', '127_a_support.npy'))

train_eval_clf(X_train[:, support], y_train, X_test[:, support], y_test)

Accuracy: 	0.522
Precision: 	0.588
Recall: 	0.400
F1-Score: 	0.476


### Sequential Feature Selection (Forward)

Moved to script sfs.py

### Mutual Information

In [343]:
mi_scores = mutual_info_classif(X_train, y_train, discrete_features=discrete_feats, n_neighbors=3)
mi_score_selected_index = np.where(mi_scores > 0.02)[0]

In [344]:
train_eval_clf(X_train[:, mi_score_selected_index], y_train, X_test[:, mi_score_selected_index], y_test)

Accuracy: 	0.457
Precision: 	0.500
Recall: 	0.320
F1-Score: 	0.390


In [345]:
selected_feats = feat_cols[mi_score_selected_index]
print(f'Selected {len(selected_feats)} features, {len(np.intersect1d(selected_feats, na_cols))} of which are from zeroed channels')

Selected 187 features, 9 of which are from zeroed channels


### Select from tree-based classifier

In [346]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(X_train, y_train)
selector = SelectFromModel(clf, max_features=300, prefit=True)

In [347]:
train_eval_clf(selector.transform(X_train), y_train, selector.transform(X_test), y_test)

Accuracy: 	0.609
Precision: 	0.684
Recall: 	0.520
F1-Score: 	0.591


In [348]:
print(f'Selected {selector.transform(X_train).shape[1]} features')

Selected 193 features


### Select from linear classifier

In [375]:
scaler = StandardScaler()
scaler.fit(X_train)
X_new = scaler.transform(X_train)

clf = svm.LinearSVC(C=1, penalty='l1', dual=False, max_iter=10000)
clf = clf.fit(X_new, y_train)
selector = SelectFromModel(clf, max_features=300, prefit=True)

In [376]:
train_eval_clf(selector.transform(X_train), y_train, selector.transform(X_test), y_test)

Accuracy: 	0.478
Precision: 	0.529
Recall: 	0.360
F1-Score: 	0.429


In [377]:
print(f'Selected {selector.transform(X_train).shape[1]} features')

Selected 130 features
